In [10]:
# edges are encoded at 8 bit bytes. The first bit encodes flipped or not. The last 4 bits encode slice (0, 1, 2) and number within the slice (0, 1, 2, 3)

edges   = [0x10, 0x11, 0x12, 0x13, 0x04, 0x05, 0x06, 0x07, 0x08, 0x09, 0x0A, 0x0B]
corners = [0x00, 0x01, 0x02, 0x03, 0x04, 0x05, 0x06, 0x07]
parity  = [0x00]
cube    = bytes(edges + corners + parity)


In [11]:
def generate_new(members, generators):
    candidates = dict()
    for name, cube in members.items():
        candidates.update({name + n: bytes([cube[0x0F & generator[i]] ^ (generator[i] & 0x80)  for i in range(0,12)] +
                                           [(cube[12 + (0x0F & generator[j])] & 0x0F) + (((cube[12 + (0x0F & generator[j])] & 0xF0) + (generator[j] & 0xF0))%0xC0)    for j in range(12, 20)] +
                                           [cube[20] ^ generator[20]])
                           for n, generator in generators.items()} )
        
            
    return candidates
    
    

In [12]:
# Define base permutations (clockwise quarter turns in each face - cubies numbered from 0
moves = {'R': bytes([0x00, 0x01, 0x06, 0x07, 0x04, 0x05, 0x03, 0x02, 0x08, 0x09, 0x0A, 0x0B] + [0x00, 0x01, 0x07, 0x06, 0x04, 0x05, 0x02, 0x03] + [0x80]),
         'L': bytes([0x04, 0x05, 0x02, 0x03, 0x01, 0x00, 0x06, 0x07, 0x08, 0x09, 0x0A, 0x0B] + [0x05, 0x04, 0x02, 0x03, 0x00, 0x01, 0x06, 0x07] + [0x80]),
         'F': bytes([0x00, 0x01, 0x02, 0x03, 0x04, 0x09, 0x08, 0x07, 0x05, 0x06, 0x0A, 0x0B] + [0x00, 0x86, 0x02, 0x84, 0x41, 0x05, 0x43, 0x07] + [0x80]),
         'B': bytes([0x00, 0x01, 0x02, 0x03, 0x0B, 0x05, 0x06, 0x0A, 0x08, 0x09, 0x04, 0x07] + [0x87, 0x01, 0x85, 0x03, 0x04, 0x40, 0x06, 0x42] + [0x80]),
         'U': bytes([0x88, 0x01, 0x8B, 0x03, 0x04, 0x05, 0x06, 0x07, 0x82, 0x09, 0x0A, 0x80] + [0x44, 0x01, 0x02, 0x47, 0x83, 0x05, 0x06, 0x80] + [0x80]),
         'D': bytes([0x00, 0x8A, 0x02, 0x89, 0x04, 0x05, 0x06, 0x07, 0x08, 0x81, 0x83, 0x0B] + [0x00, 0x45, 0x46, 0x03, 0x04, 0x82, 0x81, 0x07] + [0x80])
         }

In [13]:
mask =  bytes([0x00, 0x00, 0x00, 0xFF, 0x00, 0x00, 0x00, 0xFF, 0x00, 0x00, 0x00, 0xFF, 0x00, 0x00, 0x00, 0xFF,0x00, 0x00, 0x00, 0xFF])

In [31]:
generate_new({'':cube},moves)

b'\x10\x11\x12\x13\x04\x05\x06\x07\x08\t\n\x0b\x00\x01\x02\x03\x04\x05\x06\x07\x00'


{'R': b'\x10\x11\x06\x07\x04\x05\x13\x12\x08\t\n\x0b\x00\x01\x07\x06\x04\x05\x02\x03\x80',
 'L': b'\x04\x05\x12\x13\x11\x10\x06\x07\x08\t\n\x0b\x05\x04\x02\x03\x00\x01\x06\x07\x80',
 'F': b'\x10\x11\x12\x13\x04\t\x08\x07\x05\x06\n\x0b\x00\x86\x02\x84A\x05C\x07\x80',
 'B': b'\x10\x11\x12\x13\x0b\x05\x06\n\x08\t\x04\x07\x87\x01\x85\x03\x04@\x06B\x80',
 'U': b'\x88\x11\x8b\x13\x04\x05\x06\x07\x92\t\n\x90D\x01\x02G\x83\x05\x06\x80\x80',
 'D': b'\x10\x8a\x12\x89\x04\x05\x06\x07\x08\x91\x93\x0b\x00EF\x03\x04\x82\x81\x07\x80'}

In [28]:
squared_moves = generate_new(moves, moves)
three_moves   = generate_new(moves, squared_moves)
four_moves    = generate_new(squared_moves, squared_moves)

   

In [29]:
# Check that four moves brings youback to the start
print(four_moves['UUUU'])
print(four_moves['DDDD'])
print(four_moves['FFFF'])
print(four_moves['RRRR'])
print(four_moves['LLLL'])
print(four_moves['RRRR'])



b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x00\x01\x02\x03\x04\x05\x06\x07\x00'
b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x00\x01\x02\x03\x04\x05\x06\x07\x00'
b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x00\x01\x02\x03\x04\x05\x06\x07\x00'
b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x00\x01\x02\x03\x04\x05\x06\x07\x00'
b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x00\x01\x02\x03\x04\x05\x06\x07\x00'
b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x00\x01\x02\x03\x04\x05\x06\x07\x00'


In [27]:
print('\'R\' :bytes(' + str(moves['R']) + ')')
print('\'L\' :bytes(' + str(moves['L']) + ')')
print('\'F\' :bytes(' + str(moves['F']) + ')')
print('\'B\' :bytes(' + str(moves['B']) + ')')
print('\'U\' :bytes(' + str(moves['U']) + ')')
print('\'D\' :bytes(' + str(moves['D']) + ')')


'R' :bytes(b'\x00\x01\x06\x07\x04\x05\x03\x02\x08\t\n\x0b\x00\x01\x07\x06\x04\x05\x02\x03\x80')
'L' :bytes(b'\x04\x05\x02\x03\x01\x00\x06\x07\x08\t\n\x0b\x05\x04\x02\x03\x00\x01\x06\x07\x80')
'F' :bytes(b'\x00\x01\x02\x03\x04\t\x08\x07\x05\x06\n\x0b\x00\x86\x02\x84A\x05C\x07\x80')
'B' :bytes(b'\x00\x01\x02\x03\x0b\x05\x06\n\x08\t\x04\x07\x87\x01\x85\x03\x04@\x06B\x80')
'U' :bytes(b'\x88\x01\x8b\x03\x04\x05\x06\x07\x82\t\n\x80D\x01\x02G\x83\x05\x06\x80\x80')
'D' :bytes(b'\x00\x8a\x02\x89\x04\x05\x06\x07\x08\x81\x83\x0b\x00EF\x03\x04\x82\x81\x07\x80')


In [30]:
print('\'S\' :bytes(' + str(squared_moves['RR']) + ')')
print('\'M\' :bytes(' + str(squared_moves['LL']) + ')')
print('\'G\' :bytes(' + str(squared_moves['FF']) + ')')
print('\'C\' :bytes(' + str(squared_moves['BB']) + ')')
print('\'V\' :bytes(' + str(squared_moves['UU']) + ')')
print('\'E\' :bytes(' + str(squared_moves['DD']) + ')')


'S' :bytes(b'\x00\x01\x03\x02\x04\x05\x07\x06\x08\t\n\x0b\x00\x01\x03\x02\x04\x05\x07\x06\x00')
'M' :bytes(b'\x01\x00\x02\x03\x05\x04\x06\x07\x08\t\n\x0b\x01\x00\x02\x03\x05\x04\x06\x07\x00')
'G' :bytes(b'\x00\x01\x02\x03\x04\x06\x05\x07\t\x08\n\x0b\x00\x03\x02\x01\x06\x05\x04\x07\x00')
'C' :bytes(b'\x00\x01\x02\x03\x07\x05\x06\x04\x08\t\x0b\n\x02\x01\x00\x03\x04\x07\x06\x05\x00')
'V' :bytes(b'\x02\x01\x00\x03\x04\x05\x06\x07\x0b\t\n\x08\x03\x01\x02\x00\x07\x05\x06\x04\x00')
'E' :bytes(b'\x00\x03\x02\x01\x04\x05\x06\x07\x08\n\t\x0b\x00\x02\x01\x03\x04\x06\x05\x07\x00')


In [30]:
print('\'r\' :bytes(' + str(three_moves['RRR']) + ')')
print('\'l\' :bytes(' + str(three_moves['LLL']) + ')')
print('\'f\' :bytes(' + str(three_moves['FFF']) + ')')
print('\'b\' :bytes(' + str(three_moves['BBB']) + ')')
print('\'u\' :bytes(' + str(three_moves['UUU']) + ')')
print('\'d\' :bytes(' + str(three_moves['DDD']) + ')')



'r' :bytes(b'\x00\x01\x07\x06\x04\x05\x02\x03\x08\t\n\x0b\x00\x01\x06\x07\x04\x05\x03\x02\x80')
'l' :bytes(b'\x05\x04\x02\x03\x00\x01\x06\x07\x08\t\n\x0b\x04\x05\x02\x03\x01\x00\x06\x07\x80')
'f' :bytes(b'\x00\x01\x02\x03\x04\x08\t\x07\x06\x05\n\x0b\x00\x84\x02\x86C\x05A\x07\x80')
'b' :bytes(b'\x00\x01\x02\x03\n\x05\x06\x0b\x08\t\x07\x04\x85\x01\x87\x03\x04B\x06@\x80')
'u' :bytes(b'\x8b\x01\x88\x03\x04\x05\x06\x07\x80\t\n\x82G\x01\x02D\x80\x05\x06\x83\x80')
'd' :bytes(b'\x00\x89\x02\x8a\x04\x05\x06\x07\x08\x83\x81\x0b\x00FE\x03\x04\x81\x82\x07\x80')
